In [ ]:
%load_ext autoreload
%autoreload 2
%cd /home/ubuntu/super-resolution/

In [ ]:
import json

import numpy as np
import pandas as pd
from fs_s3fs import S3FS

import wandb
from hrnet.src.train import main
import torchvision
from sr.niva_models import TorchUnetv2
from torch import nn 
import os 
from types import SimpleNamespace



aws_access_key_id = ''
aws_secret_access_key = '

filesystem = S3FS(
    bucket_name='',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key, 
    region='eu-central-1')

country_norm_df = pd.read_parquet(filesystem.openbin('metadata/s2_norm_per_country.pq'))

data_df = pd.read_parquet(filesystem.openbin('metadata/npz_info_small.pq'))
data_df.reset_index(inplace=True)

scores_df = pd.read_parquet(filesystem.openbin('baseline-scores-sr/scores-bicubic-32x32-2p5m-hm.pq')
                            ).rename(columns={'name': 'singleton_npz_filename'})

data_df = pd.merge(data_df, scores_df, on='singleton_npz_filename')
data_df['MSE_ratio'] = data_df['MSE_s']/data_df['MSE_s_c']

In [ ]:
import torch

### Data setup

In [ ]:
norm_deimos = {k: v for k, v in np.load(filesystem.openbin('metadata/deimos_min_max_norm.npz')).items()}
norm_s2 = {k: v for k, v in np.load(filesystem.openbin('metadata/s2_min_max_norm.npz')).items()}

# Filter data

In [ ]:
filtered_data = data_df[(data_df['SSIM_s_c'] > .2) &
                        (data_df['PSNR_s_c'] > 10) &
                        (data_df['MSE_ratio'] < 10) &
                        (data_df['is_shadow_v2'] == False) &
                        (data_df['countries'] == 'Lithuania') &
                        (data_df['num_tstamps'] > 1)]

In [ ]:
len(filtered_data)

###  Wandb setup

In [ ]:
# ! wandb login 

# Training

In [ ]:
wandb.init(project='', entity='', config=config)

In [ ]:
with open('input/config-local-hrn-pix-shu.json') as f:
    config = json.load(f)

In [ ]:
MODEL_DIR = ''
with open(os.path.join(MODEL_DIR, 'model_cfg.json'), 'r') as jfile:
    model_cfg = json.load(jfile)
    
perceptual_model = TorchUnetv2(4, config=SimpleNamespace(**model_cfg))
perceptual_model.load_state_dict(torch.load(os.path.join(MODEL_DIR, 'model.pth')))
perceptual_model.eval()
perceptual_model.to(torch.device('cuda'))

# Perceptual loss model 

In [ ]:
model = main(config,
             filtered_data,
             normalize=True,
             country_norm_df=country_norm_df,
             norm_deimos_npz=norm_deimos,
             norm_s2_npz=norm_s2, perceptual_loss_model=perceptual_model)